### Simple Attendance Tracker using Python

In [3]:
import pandas as pd

# Sample data for 10 students
students_data = {
    "Roll Number": [f"00{i+1}" for i in range(11)],
    "Student Mail ID": [f"student{i+1}@example.com" for i in range(11)],
    "Subject 1": ["Math", "English", "Physics", "History", "Biology", "Math", "Chemistry", "Economics", "English", "Geography","AI"],
    "Subject 2": ["Science", "Math", "Chemistry", "Biology", "History", "English", "Physics", "Math", "Biology", "Civics","AI"],
    "Subject 3": ["English", "Science", "Math", "Civics", "Physics", "History", "Biology", "Chemistry", "Economics", "Math","AI"]
}

# Create DataFrame
students_df = pd.DataFrame(students_data)

# Save to Excel
excel_file_path = "Sample_Students_Data.xlsx"
students_df.to_excel(excel_file_path, index=False)

excel_file_path


'Sample_Students_Data.xlsx'

In [4]:
! pip install pandas openpyxl


In [5]:
import openpyxl

wb = openpyxl.load_workbook('Sample_Students_Data.xlsx')
sheet = wb.active
print(sheet.cell(row=2, column=1).value)  # Example to print data

001


In [6]:
import pandas as pd
df = pd.read_excel('Sample_Students_Data.xlsx')
df.head()


,Roll Number,Student Mail ID,Subject 1,Subject 2,Subject 3
0,1,student1@example.com,Math,Science,English
1,2,student2@example.com,English,Math,Science
2,3,student3@example.com,Physics,Chemistry,Math
3,4,student4@example.com,History,Biology,Civics
4,5,student5@example.com,Biology,History,Physics


In [7]:
! pip install openpyxl ttkbootstrap


In [16]:
import openpyxl
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import tkinter as tk
from tkinter import messagebox

# Load workbook and sheet
book = openpyxl.load_workbook('D:\\Anand Kumar\\OneDrive\\Desktop\\Book1.xlsx')

sheet = book['Sheet1']

# Constants
STAFF_EMAILS = ['ak6491553@gmail.com', 'gulsonyadav@gmail.com']  # Update as needed
SUBJECTS = {1: "CI", 2: "Python", 3: "DM"}
WARNINGS = {
    1: "Warning!!! You can take only one more day leave for CI class.",
    2: "Warning!!! You can take only one more day leave for Python class.",
    3: "Warning!!! You can take only one more day leave for DM class."
}
FROM_ID = 'ak6491553@gmail.com'
PASSWORD = 'ANAND12345'  # Consider using environment variables instead

# Save file function
def save_file():
    book.save('Sample_Students_Data.xlsx')

# Email sending function
def send_email(to_list, subject, body):
    try:
        server = smtplib.SMTP('smtp.gmail.com', 587, timeout=120)
        server.starttls()
        server.login(FROM_ID, PASSWORD)

        for to_email in to_list:
            message = MIMEMultipart()
            message['Subject'] = subject
            message.attach(MIMEText(body, 'plain'))
            server.sendmail(FROM_ID, to_email, message.as_string())

        server.quit()
    except Exception as e:
        print(f"Error sending email: {e}")

# Check threshold and trigger emails
def check_threshold(absent_counts, row_nums, subject_index, log_widget):
    student_emails_warn = []
    student_emails_lack = []
    roll_lack_list = []

    for count, row in zip(absent_counts, row_nums):
        email = sheet.cell(row=row, column=2).value
        roll_no = sheet.cell(row=row, column=1).value

        if count == 2:
            student_emails_warn.append(email)
        elif count > 2:
            student_emails_lack.append(email)
            roll_lack_list.append(str(roll_no))

    if student_emails_warn:
        send_email(student_emails_warn, "Attendance Warning", WARNINGS[subject_index])
        log_widget.insert(tk.END, f"Warning email sent to: {', '.join(student_emails_warn)}\n")

    if student_emails_lack:
        subject_name = SUBJECTS[subject_index]
        msg_students = f"You have lack of attendance in {subject_name}!"
        msg_staff = f"The following students have lack of attendance in {subject_name}: {', '.join(roll_lack_list)}"

        send_email(student_emails_lack, "Lack of Attendance", msg_students)
        send_email([STAFF_EMAILS[subject_index - 1]], "Lack of Attendance Report", msg_staff)
        log_widget.insert(tk.END, f"Lack emails sent to students and staff for: {subject_name}\n")

# Mark absentees function (Submit)
def mark_absentees():
    try:
        subject_index = int(subject_dropdown.get().split("-")[0])
        roll_numbers = list(map(int, entry_rolls.get().strip().split()))

        row_nums = []
        absent_counts = []

        for roll in roll_numbers:
            for i in range(2, sheet.max_row + 1):
                if sheet.cell(row=i, column=1).value == roll:
                    col = subject_index + 2
                    count = sheet.cell(row=i, column=col).value or 0
                    count += 1
                    sheet.cell(row=i, column=col).value = count
                    absent_counts.append(count)
                    row_nums.append(i)
                    break

        save_file()
        log_box.insert(tk.END, f"Marked absent: {', '.join(map(str, roll_numbers))}\n")
        log_box.insert(tk.END, "Attendance updated in Excel.\n")
        check_threshold(absent_counts, row_nums, subject_index, log_box)

    except Exception as e:
        messagebox.showerror("Error", str(e))

# Show absentees for selected subject
def show_absentees():
    try:
        subject_index = int(subject_dropdown.get().split("-")[0])
        col = subject_index + 2
        absentees = []

        for i in range(2, sheet.max_row + 1):
            roll = sheet.cell(row=i, column=1).value
            name = sheet.cell(row=i, column=3).value
            count = sheet.cell(row=i, column=col).value or 0
            if count > 0:
                absentees.append(f"{roll} - {name} ({count} absences)")

        if absentees:
            log_box.insert(tk.END, f"\nAbsentees for {SUBJECTS[subject_index]}:\n")
            log_box.insert(tk.END, "\n".join(absentees) + "\n")
        else:
            log_box.insert(tk.END, f"\nNo absentees yet for {SUBJECTS[subject_index]}.\n")
    except Exception as e:
        messagebox.showerror("Error", str(e))

# GUI Setup
root = tk.Tk()
root.title("Attendance Management System")
root.geometry("700x500")
root.configure(bg="#fef3c7")  # light warm yellow

# Heading
tk.Label(root, text="Student Attendance Manager", font=("Arial", 18, "bold"), bg="#fef3c7", fg="#7c3aed").pack(pady=10)

# Subject selection
tk.Label(root, text="Select Subject", font=("Arial", 12), bg="#fef3c7").pack()
subject_dropdown = tk.StringVar()
subject_dropdown.set("1 - CI")
tk.OptionMenu(root, subject_dropdown, *["1 - CI", "2 - Python", "3 - DM"]).pack(pady=5)

# Roll number input
tk.Label(root, text="Enter Roll Numbers (space-separated)", font=("Arial", 12), bg="#fef3c7").pack()
entry_rolls = tk.Entry(root, font=("Arial", 12), width=40)
entry_rolls.pack(pady=5)

# Submit button
submit_btn = tk.Button(root, text="Submit Absentees", command=mark_absentees, bg="#4f46e5", fg="white", font=("Arial", 12), relief="raised")
submit_btn.pack(pady=10)

# Show absentees button
view_btn = tk.Button(root, text="Show Absentees", command=show_absentees, bg="#10b981", fg="white", font=("Arial", 12), relief="raised")
view_btn.pack(pady=5)

# Log box for status and feedback
log_box = tk.Text(root, height=10, width=70, font=("Arial", 10), bg="#f0fdf4")
log_box.pack(pady=10)

# Main loop
root.mainloop()